## **601. Human Traffic of Stadium**
https://leetcode.com/problems/human-traffic-of-stadium/description/


In [17]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, DateType
from datetime import datetime

# Initialize Spark Session
spark = SparkSession.builder.appName("CreateTable").getOrCreate()

# Define Schema
schema = StructType([
    StructField("id", IntegerType(), False),
    StructField("visit_date", DateType(), False),
    StructField("people", IntegerType(), False)
])

# Sample Data
data = [
    (1, datetime.strptime("2017-01-01", "%Y-%m-%d").date(), 10),
    (2, datetime.strptime("2017-01-02", "%Y-%m-%d").date(), 109),
    (3, datetime.strptime("2017-01-03", "%Y-%m-%d").date(), 150),
    (4, datetime.strptime("2017-01-04", "%Y-%m-%d").date(), 99),
    (5, datetime.strptime("2017-01-05", "%Y-%m-%d").date(), 145),
    (6, datetime.strptime("2017-01-06", "%Y-%m-%d").date(), 1455),
    (7, datetime.strptime("2017-01-07", "%Y-%m-%d").date(), 199),
    (8, datetime.strptime("2017-01-09", "%Y-%m-%d").date(), 188)
]


# Create DataFrame
df = spark.createDataFrame(data, schema=schema)

# Register as Table
df.createOrReplaceTempView("stadium")

# Verify Data
spark.sql("SELECT * FROM stadium").show()


+---+----------+------+
| id|visit_date|people|
+---+----------+------+
|  1|2017-01-01|    10|
|  2|2017-01-02|   109|
|  3|2017-01-03|   150|
|  4|2017-01-04|    99|
|  5|2017-01-05|   145|
|  6|2017-01-06|  1455|
|  7|2017-01-07|   199|
|  8|2017-01-09|   188|
+---+----------+------+



In [19]:
filter_query = """
CREATE OR REPLACE TEMP VIEW consecutive AS
SELECT 
    *, 
    id - ROW_NUMBER() OVER (ORDER BY id) AS diff
FROM stadium 
WHERE people >= 100
"""
spark.sql(filter_query)

# Show Consecutive Data
spark.sql("SELECT * FROM consecutive").show()


25/02/16 11:07:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/16 11:07:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/16 11:07:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+---+----------+------+----+
| id|visit_date|people|diff|
+---+----------+------+----+
|  2|2017-01-02|   109|   1|
|  3|2017-01-03|   150|   1|
|  5|2017-01-05|   145|   2|
|  6|2017-01-06|  1455|   2|
|  7|2017-01-07|   199|   2|
|  8|2017-01-09|   188|   2|
+---+----------+------+----+



In [23]:
query3 = """CREATE OR REPLACE TEMP VIEW rnk_view AS

    SELECT *, COUNT(*) OVER (PARTITION BY diff) AS rnk
    FROM consecutive
"""
df_with_rnk = spark.sql(query3)
spark.sql("SELECT * FROM rnk_view").show()


25/02/16 11:11:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/16 11:11:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+---+----------+------+----+---+
| id|visit_date|people|diff|rnk|
+---+----------+------+----+---+
|  2|2017-01-02|   109|   1|  2|
|  3|2017-01-03|   150|   1|  2|
|  5|2017-01-05|   145|   2|  4|
|  6|2017-01-06|  1455|   2|  4|
|  7|2017-01-07|   199|   2|  4|
|  8|2017-01-09|   188|   2|  4|
+---+----------+------+----+---+



25/02/16 11:11:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/16 11:11:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [22]:
query4 = """
SELECT id, visit_date, people  
FROM rnk_view
WHERE rnk >= 3
ORDER BY id
"""
df_result = spark.sql(query4)
df_result.show()

25/02/16 11:10:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/16 11:10:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/16 11:10:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/16 11:10:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+---+----------+------+
| id|visit_date|people|
+---+----------+------+
|  5|2017-01-05|   145|
|  6|2017-01-06|  1455|
|  7|2017-01-07|   199|
|  8|2017-01-09|   188|
+---+----------+------+



## **262. Trips and Users**
https://leetcode.com/problems/trips-and-users/description/

In [25]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType
from datetime import datetime

spark = SparkSession.builder.appName("TaxiTrips").getOrCreate()

# Define Schema for 'Trips' Table
trips_schema = StructType([
    StructField("id", IntegerType(), False),
    StructField("client_id", IntegerType(), False),
    StructField("driver_id", IntegerType(), False),
    StructField("city_id", IntegerType(), False),
    StructField("status", StringType(), False),
    StructField("request_at", StringType(), False)
])

# Sample Data for 'Trips' Table
trips_data = [
    (1, 1, 10, 1, "completed", "2013-10-01"),
    (2, 2, 11, 1, "cancelled_by_driver", "2013-10-01"),
    (3, 3, 12, 6, "completed", "2013-10-01"),
    (4, 4, 13, 6, "cancelled_by_client", "2013-10-01"),
    (5, 1, 10, 1, "completed", "2013-10-02"),
    (6, 2, 11, 6, "completed", "2013-10-02"),
    (7, 3, 12, 6, "completed", "2013-10-02"),
    (8, 2, 12, 12, "completed", "2013-10-03"),
    (9, 3, 10, 12, "completed", "2013-10-03"),
    (10, 4, 13, 12, "cancelled_by_driver", "2013-10-03")
]

# Create DataFrame for 'Trips' table
df_trips = spark.createDataFrame(trips_data, schema=trips_schema)

# Register as SQL Table
df_trips.createOrReplaceTempView("Trips")

# Show 'Trips' Table
df_trips.show()

+---+---------+---------+-------+-------------------+----------+
| id|client_id|driver_id|city_id|             status|request_at|
+---+---------+---------+-------+-------------------+----------+
|  1|        1|       10|      1|          completed|2013-10-01|
|  2|        2|       11|      1|cancelled_by_driver|2013-10-01|
|  3|        3|       12|      6|          completed|2013-10-01|
|  4|        4|       13|      6|cancelled_by_client|2013-10-01|
|  5|        1|       10|      1|          completed|2013-10-02|
|  6|        2|       11|      6|          completed|2013-10-02|
|  7|        3|       12|      6|          completed|2013-10-02|
|  8|        2|       12|     12|          completed|2013-10-03|
|  9|        3|       10|     12|          completed|2013-10-03|
| 10|        4|       13|     12|cancelled_by_driver|2013-10-03|
+---+---------+---------+-------+-------------------+----------+



In [26]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# Define Schema for 'Users' Table
users_schema = StructType([
    StructField("users_id", IntegerType(), False),
    StructField("banned", StringType(), False),
    StructField("role", StringType(), False)
])

# Sample Data for 'Users' Table
users_data = [
    (1, "No", "client"),
    (2, "Yes", "client"),
    (3, "No", "client"),
    (4, "No", "client"),
    (10, "No", "driver"),
    (11, "No", "driver"),
    (12, "No", "driver"),
    (13, "No", "driver")
]

# Create DataFrame for 'Users' table
df_users = spark.createDataFrame(users_data, schema=users_schema)

# Register as SQL Table
df_users.createOrReplaceTempView("Users")

# Show 'Users' Table
df_users.show()

+--------+------+------+
|users_id|banned|  role|
+--------+------+------+
|       1|    No|client|
|       2|   Yes|client|
|       3|    No|client|
|       4|    No|client|
|      10|    No|driver|
|      11|    No|driver|
|      12|    No|driver|
|      13|    No|driver|
+--------+------+------+



### solution 1 (longer 3 queries)

In [41]:
query = """
CREATE OR REPLACE TEMP VIEW trips_no_banned_view AS
select t.id, t.client_id, u.banned AS client_banned, t.driver_id, u2.banned as driver_banned, t.city_id, t.status, t.request_at
from Trips t
join Users u on t.client_id = u.users_id 
join Users u2 on t.driver_id = u2.users_id
where u.banned = 'No' and u2.banned = 'No'
"""

spark.sql(query)
spark.sql("SELECT * FROM trips_no_banned_view order by id").show()

+---+---------+-------------+---------+-------------+-------+-------------------+----------+
| id|client_id|client_banned|driver_id|driver_banned|city_id|             status|request_at|
+---+---------+-------------+---------+-------------+-------+-------------------+----------+
|  1|        1|           No|       10|           No|      1|          completed|2013-10-01|
|  3|        3|           No|       12|           No|      6|          completed|2013-10-01|
|  4|        4|           No|       13|           No|      6|cancelled_by_client|2013-10-01|
|  5|        1|           No|       10|           No|      1|          completed|2013-10-02|
|  7|        3|           No|       12|           No|      6|          completed|2013-10-02|
|  9|        3|           No|       10|           No|     12|          completed|2013-10-03|
| 10|        4|           No|       13|           No|     12|cancelled_by_driver|2013-10-03|
+---+---------+-------------+---------+-------------+-------+---------

In [45]:
query2 = """
create or replace temp view trips_groupby_status as
select request_at, status, count(status) as count_status
from trips_no_banned_view
group by request_at, status
order by request_at"""

spark.sql(query2)

spark.sql("select * from trips_groupby_status").show()

+----------+-------------------+------------+
|request_at|             status|count_status|
+----------+-------------------+------------+
|2013-10-01|          completed|           2|
|2013-10-01|cancelled_by_client|           1|
|2013-10-02|          completed|           2|
|2013-10-03|          completed|           1|
|2013-10-03|cancelled_by_driver|           1|
+----------+-------------------+------------+



In [ ]:
query3 = """
select 
    request_at, 
    SUM(CASE WHEN status IN ('cancelled_by_client', 'cancelled_by_driver') THEN count_status ELSE 0 END) / sum(count_status) AS cancellation_rate
from trips_groupby_status
group by request_at
"""

spark.sql(query3).show()

AnalysisException: [MISSING_AGGREGATION] The non-aggregating expression "status" is based on columns which are not participating in the GROUP BY clause.
Add the columns or the expression to the GROUP BY, aggregate the expression, or use "any_value(status)" if you do not care which of the values within a group is returned.;
Aggregate [request_at#1177], [request_at#1177, (cast(CASE WHEN status#1178 IN (cancelled_by_client,cancelled_by_driver) THEN count_status#1179L ELSE cast(0 as bigint) END as double) / cast(sum(count_status#1179L) as double)) AS cancellation_rate#1175]
+- SubqueryAlias trips_groupby_status
   +- View (`trips_groupby_status`, [request_at#1177,status#1178,count_status#1179L])
      +- Project [cast(request_at#1189 as string) AS request_at#1177, cast(status#1188 as string) AS status#1178, cast(count_status#1176L as bigint) AS count_status#1179L]
         +- Sort [request_at#1189 ASC NULLS FIRST], true
            +- Aggregate [request_at#1189, status#1188], [request_at#1